# Workshop 1 - Question and Answers
In this workshop, you will learning how to write prompts and feed them into LLMs. You
will also be learning how to use different prompt techniques to improve the response
from the LLM.

## Loading and Explorng the Dataset
The workshop will be using [`facebook/ExploreToM`](https://huggingface.co/datasets/facebook/ExploreToM) dataset from [HuggingFace](https://huggingface.co).

In [1]:
# TODO: Load the following libraries: datasets
from datasets import load_dataset

In [4]:
# Dataset name
dataset_name = "facebook/ExploreToM"
 
# TODO: load and explore the dataset
ds = load_dataset(dataset_name)

In [8]:
# TODO: number of rows in the dataset
print(ds.shape)

# TODO: Keys in the dataset
print(ds.keys())

# TODO: Feature names
print(ds['train'].features)

# TODO: Display a single row
idx = 10
for k, v in ds['train'][idx].items():
   print(f"key: {k}, value: {v}")


{'train': (13309, 18)}
dict_keys(['train'])
{'story_structure': Value('string'), 'infilled_story': Value('string'), 'question': Value('string'), 'expected_answer': Value('string'), 'qprop=params': Value('string'), 'qprop=nth_order': Value('int64'), 'qprop=non_unique_mental_state': Value('bool'), 'sprop=is_false_belief_story_1st': Value('bool'), 'sprop=is_false_belief_story_1st_and_2nd': Value('bool'), 'sprop=story_accuracy_1st_raw': Value('float64'), 'sprop=story_accuracy_1st_infilled': Value('float64'), 'sprop=global_idx': Value('int64'), 'param=story_type': Value('string'), 'param=num_stories_total': Value('int64'), 'param=max_sentences': Value('int64'), 'param=num_people': Value('int64'), 'param=num_moves': Value('int64'), 'param=num_rooms': Value('int64')}
key: story_structure, value: Dylan entered the green room. Dylan moved the pocket watch to the wooden chest, which is also located in the green room. While this action was happening, Clayton witnessed this action in secret (and o

In [10]:
# TODO: import pipeline
from transformers import pipeline

## `pipeline`
[`pipeline`](https://huggingface.co/docs/transformers/en/main_classes/pipelines) is an easy to use API to perform inferencing. It provides a wrapper for task-specific pipelines and abstracts most of the complexity by allowing you to focus on the model and the task. 

You can use `pipeline` to perform summarisation, image classification, audio generation, etc. You can find an exhaustive list of `pipeline` task [here](https://huggingface.co/docs/transformers/en/main_classes/pipelines#transformers.pipeline.task).

In [11]:
# TODO: Summarise the text with the pipeline's default model
qna = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [17]:
idx = 100
story = ds['train'][idx]['story_structure']
story_infilled = ds['train'][idx]['infilled_story']
question = ds['train'][idx]['question']
expected_answer = ds['train'][idx]['expected_answer']

In [18]:
# question = the question we are asking
# context = story
result = qna(question=question, context=story_infilled)

print(f'question: {question}')
print(f'story: {story}')
print(f'predicted: {result}')
print(f'actual: {expected_answer}')

question: What does Isabella think about Colton's belief on festival marketing strategies? (knows about it / does not know about it)
story: Leslie entered the main tent. Leslie left the main tent. Isabella entered the storage trailer. Isabella moved the stuffed rabbit to the wooden chest, which is also located in the storage trailer. Leslie entered the main tent. Isabella moved the stuffed rabbit to the main tent, leaving the wooden chest in its original location. Isabella told out loud about the festival marketing strategies. Isabella told privately to Colton that Leslie is in the main tent. While this action was happening, Leslie witnessed this action in secret (and only this action).
predicted: {'score': 0.20097941160202026, 'start': 1739, 'end': 1765, 'answer': 'Isabella mouthed a message'}
actual: does not know about it


## Manual Inference - Question and Answer
In this section, we will look at what `pipeline` does under the hood to perform its inference. This will give us a better understanding of the major steps involved.

In [19]:
# TODO: load tokenizer
from transformers import AutoTokenizer

## DistilBERT base cased distilled SQuAD
DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. More details [here](https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).

In [20]:
model_name = "distilbert/distilbert-base-cased-distilled-squad"

In [21]:
# TODO: Create a tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [27]:
# TODO: Encode text
message = "big black bug bleeds black blood"
#message = "A survey of PostgreSQL users has found that the levels of uptime experienced using cloud providers falls well short of their expectations in terms of reliability."

# encode the text
enc_text = tokenizer(message, return_tensors='pt')

print(enc_text)


{'input_ids': tensor([[  101,  1992,  1602, 15430, 24752,  1116,  1602,  1892,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [28]:
for i in enc_text.input_ids[0]:
   print(i, tokenizer.decode(i))

tensor(101) [CLS]
tensor(1992) big
tensor(1602) black
tensor(15430) bug
tensor(24752) bleed
tensor(1116) ##s
tensor(1602) black
tensor(1892) blood
tensor(102) [SEP]


In [29]:
# TODO: Encoding multiple texts
messages = [
   'big black bug bleeds black blood',
   'she sell sea shells on the sea shore',
   'A long time ago, in a galaxy far far away',
   'Thou shalt not make a machine in the likeness of a human mind.'
]

enc_texts = tokenizer(messages, return_tensors='pt', padding=True)

print(enc_texts)


{'input_ids': tensor([[  101,  1992,  1602, 15430, 24752,  1116,  1602,  1892,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  1131,  4582,  2343, 11251,  1113,  1103,  2343,  5781,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,   138,  1263,  1159,  2403,   117,  1107,   170, 15593,  1677,
          1677,  1283,   102,     0,     0,     0,     0,     0,     0],
        [  101,   157, 14640,   188, 24537,  1136,  1294,   170,  3395,  1107,
          1103,  1176,  1757,  1104,   170,  1769,  1713,   119,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [31]:
# TODO: Decode text
print(tokenizer.decode(enc_texts.input_ids[3], skip_special_tokens=True))

Thou shalt not make a machine in the likeness of a human mind.


In [32]:
# Tokenize the question and the story
enc_question = tokenizer(question, return_tensors='pt').input_ids
enc_story = tokenizer(story, return_tensors='pt').input_ids

print(enc_question)
print(enc_story)

tensor([[  101,  1327,  1674, 10206,  1341,  1164, 23065,   112,   188,  6369,
          1113,  3782,  6213, 10700,   136,   113,  3520,  1164,  1122,   120,
          1674,  1136,  1221,  1164,  1122,   114,   102]])
tensor([[  101,  8521,  2242,  1103,  1514,  9459,   119,  8521,  1286,  1103,
          1514,  9459,   119, 10206,  2242,  1103,  5092,  9404,   119, 10206,
          1427,  1103, 12084, 16225,  1106,  1103,  4122,  2229,   117,  1134,
          1110,  1145,  1388,  1107,  1103,  5092,  9404,   119,  8521,  2242,
          1103,  1514,  9459,   119, 10206,  1427,  1103, 12084, 16225,  1106,
          1103,  1514,  9459,   117,  2128,  1103,  4122,  2229,  1107,  1157,
          1560,  2450,   119, 10206,  1500,  1149,  4632,  1164,  1103,  3782,
          6213, 10700,   119, 10206,  1500,  9045,  1106, 23065,  1115,  8521,
          1110,  1107,  1103,  1514,  9459,   119,  1799,  1142,  2168,  1108,
          5664,   117,  8521,  9491,  1142,  2168,  1107,  3318,   113,

## Working with LLMs
Create and instance of the Large Language Model (LLM). We will then create a simple
prompt, tokenize the prompt and feed the tokenized prompt to the LLM. The response
from the LLM will be decoded to human friendly text.

In [33]:
# TODO: Load libraries
from transformers import AutoModelForQuestionAnswering

In [34]:
# TODO: Load question answer model
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


In [ ]:
# TODO: Encode context and question
# see above


In [36]:
# TODO: Tokenize the inputs
inputs = tokenizer(question, story, return_tensors='pt', padding=True)
print(inputs)


{'input_ids': tensor([[  101,  1327,  1674, 10206,  1341,  1164, 23065,   112,   188,  6369,
          1113,  3782,  6213, 10700,   136,   113,  3520,  1164,  1122,   120,
          1674,  1136,  1221,  1164,  1122,   114,   102,  8521,  2242,  1103,
          1514,  9459,   119,  8521,  1286,  1103,  1514,  9459,   119, 10206,
          2242,  1103,  5092,  9404,   119, 10206,  1427,  1103, 12084, 16225,
          1106,  1103,  4122,  2229,   117,  1134,  1110,  1145,  1388,  1107,
          1103,  5092,  9404,   119,  8521,  2242,  1103,  1514,  9459,   119,
         10206,  1427,  1103, 12084, 16225,  1106,  1103,  1514,  9459,   117,
          2128,  1103,  4122,  2229,  1107,  1157,  1560,  2450,   119, 10206,
          1500,  1149,  4632,  1164,  1103,  3782,  6213, 10700,   119, 10206,
          1500,  9045,  1106, 23065,  1115,  8521,  1110,  1107,  1103,  1514,
          9459,   119,  1799,  1142,  2168,  1108,  5664,   117,  8521,  9491,
          1142,  2168,  1107,  3318,  

In [37]:
# Ensure minimum and maximum token length in the answer
def ensure_size(input_ids, answer, min_length = 2, max_length = 5):
   ans_start = torch.argmax(answer['start_logits'])
   ans_end = torch.argmax(answer['end_logits']) + 1
   ans_length = ans_end - ans_start
   if ans_length < min_length:
      ans_end = min(ans_start + min_length, len(input_ids[0]))
   elif ans_length > max_length:
      ans_end = ans_start + max_length
   ans_start = max(0, ans_start)
   ans_end = min(len(input_ids[0]), ans_end)
   return (ans_start, ans_end)

In [38]:
# TODO Return a minimum of 5 tokens
result = model(inputs.input_ids, inputs.attention_mask)

In [39]:
print(result)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-3.9868, -4.3144, -5.9302, -5.6361, -4.8974, -6.5077, -6.4738, -8.0060,
         -7.9933, -5.7121, -7.4480, -6.7496, -7.1584, -7.6161, -5.2428, -1.9160,
         -0.5701, -4.5180, -4.3380, -2.8144, -0.8237, -2.0957, -3.7787, -5.4427,
         -4.1000, -5.0644, -6.5861, -2.4138, -5.7575, -5.9004, -5.2392, -5.9180,
         -6.2209, -2.7861, -5.7684, -6.2824, -5.9315, -6.4277, -6.5801, -3.0058,
         -5.6845, -6.0471, -5.0800, -6.3556, -6.6277, -2.7128, -4.6214, -6.0810,
         -5.0485, -5.9186, -7.3016, -6.3458, -5.3029, -6.4247, -7.8703, -7.3378,
         -8.0731, -7.9633, -7.5573, -7.3166, -6.8668, -5.7800, -6.7769, -6.6220,
         -2.9940, -5.8296, -5.9024, -5.5028, -6.2005, -5.8419, -0.3613, -4.1207,
         -5.9932, -5.0716, -6.0553, -6.8730, -5.9209, -5.3329, -6.0344, -6.6160,
         -3.9760, -5.6062, -5.0829, -6.2641, -6.6954, -6.2566, -6.0579, -6.0710,
         -2.7393,  4.9249,  5.7277,  6.2308,  3.2548, -0

In [43]:
import torch
# find the start and end position
ans_start = torch.argmax(result.start_logits)
ans_end = torch.argmax(result.end_logits) + 1

# extract the answer
enc_answer =inputs.input_ids[0][ans_start: ans_end]

# decode it
answer = tokenizer.decode(enc_answer)

print(answer)

out loud


In [ ]:
# TODO: Try this your self

context = """
Dickens wrote A Christmas Carol during a period when the British were exploring and re-evaluating past Christmas traditions, 
including carols, and newer customs such as cards and Christmas trees. He was influenced by the experiences of his own youth and 
by the Christmas stories of other authors, including Washington Irving and Douglas Jerrold. Dickens had written three Christmas 
stories prior to the novella, and was inspired following a visit to the Field Lane Ragged School, one of several establishments for 
London's street children. The treatment of the poor and the ability of a selfish man to redeem himself by transforming into a more 
sympathetic character are the key themes of the story. There is discussion among academics as to whether this is a fully secular 
story or a Christian allegory.
"""

question = "How many Christmas stories has Dickens wrote?"

